In [1]:

from pathlib import Path
import pandas as pd
import glob
import os,sys
import numpy as np
dir = Path('')
sys.path.append("../../")
sys.path.append("../")
sys.path.append("/home/chaki/Projects/gods_eye/")


In [2]:
path = "/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in"
directories = os.listdir(path)
directories = [os.path.join(path, dir) for dir in directories]

In [85]:
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np
sns.set(style="white") 
%matplotlib inline
def imshow(df, id):
    img_path = df[df["index"] == 2]["path"].values[0]
    img = cv2.imread(img_path)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


In [4]:
directories = os.listdir("/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in")#["images/12_people_min_samples"]

In [5]:
directories = [os.path.join("/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in", dir) for dir in directories]

In [6]:
import core.pipeline.tools as tools


In [132]:

from core.recognition.openvinoreid import OpenVinoReidentifier
from core.commons.pose_estimator_openvino import HeadPosEstimator
import core.pipeline.tools as tools

import cv2

reid = OpenVinoReidentifier("/home/chaki/Projects/gods_eye/core/weights/intel/face-reidentification-retail-0095/FP16/face-reidentification-retail-0095")
headpose = HeadPosEstimator("/home/chaki/Projects/gods_eye/core/weights/intel/whenet/FP32/whenet_224x224.xml")
def extract_features(path):
    features = []
    paths = []
    ids = []
    alignment_error = []
    for directory in path:
        if not os.path.isdir(directory):
            continue
        id = directory.split("/")[-1]
        for image in os.listdir(directory):
            image_path = os.path.join(directory,image)
            if image_path.endswith(".jpg"):
                image = cv2.imread(image_path)
                headpose.process(image)
                features.append(reid.encode(image)) 
                if headpose.job_finished():
                    # ? if condition checks if the job is finished - might be replaced by 'while'
                    head_pose_output = headpose.get_output()
                    # ? A function that returns the error based on headpose mainly, powered by blur and size errors
                    error = tools.compute_errors(
                        image, head_pose_output, None, None
                    )
                
                    # ? A function that returns the error based on head_pose mainly, powered by blur and size errors
                paths.append(image_path)
                ids.append(id)
                alignment_error.append(error)
    
    return ids, features, paths, alignment_error

id, features, paths, alignment_error = extract_features(directories)

features = [[*feature] for feature in features]
data = {'id': id,
        'path':paths,
        'features_openvino': features,
        'alignment_error': alignment_error
        }
 
# Create DataFrame
openvino = pd.DataFrame(data)
openvino.to_csv("/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in/openvino_data.csv")


/home/chaki/Projects/gods_eye/core/commons/pose_estimator_openvino.py:108: DeprecationWarning:

Reading network using constructor is deprecated. Please, use IECore.read_network() method instead

/home/chaki/Projects/gods_eye/core/commons/pose_estimator_openvino.py:118: DeprecationWarning:

'inputs' property of IENetwork class is deprecated. To access DataPtrs user need to use 'input_data' property of InputInfoPtr objects which can be accessed by 'input_info' property.

/home/chaki/Projects/gods_eye/core/commons/pose_estimator_openvino.py:169: DeprecationWarning:

'outputs' property of InferRequest is deprecated. Please instead use 'output_blobs' property.



In [217]:
# Create DataFrame
magface = pd.read_csv("/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in/data.csv")
openvino = pd.read_csv("/home/chaki/Projects/gods_eye/output/12_04-21_11-25-09/in/openvino_data.csv")

In [134]:
dlib_data = pd.read_csv("face_recognition_data.csv")
data_features_dlib = list(dlib_data.features_openvino.values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])
indexes = [len(value) != 0 for value in data_features_dlib]
dlib_data = dlib_data[indexes]
dlib_data = dlib_data.drop('Unnamed: 0', 1)
data_features_dlib = list(dlib_data.features_openvino.values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])
dlib_data["path"] = dlib_data["path"].str.replace('\\','/')


/tmp/ipykernel_84718/528420230.py:3: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [142]:
dlib_data.columns = ["id", "path", "feature_dlib"]
magface.columns = ["id", "path", "feature_512"]

first_time = True
for dir in directories:
    csv_file = os.path.join(dir,"data.csv")
    data = pd.read_csv(csv_file)
    if first_time:
        fr = data
        first_time = False
    else:
        fr = pd.concat([fr, data], ignore_index=False)
    # for df_path in dataframe:
    #     df_path = os.path.join(dir, df_path)
    #     data = pd.read_csv(df_path)
    #     break


    

In [159]:
test = dlib_data["path"].replace("\", "/")

In [225]:
dlib_data

,id,path,feature_dlib
1,person_0,in/person_0/100.jpg,"[-0.07003957778215408, 0.07996471226215363, 0...."
2,person_0,in/person_0/1007.jpg,"[-0.07282204926013947, 0.036366019397974014, 0..."
3,person_0,in/person_0/1016.jpg,"[-0.06365539878606796, 0.054096899926662445, 0..."
4,person_0,in/person_0/102.jpg,"[-0.039636239409446716, 0.06792464107275009, 0..."
5,person_0,in/person_0/1026.jpg,"[-0.038284990936517715, 0.04821193590760231, 0..."
...,...,...,...
1396,person_8,in/person_8/277.jpg,"[-0.10293488949537277, 0.1258092075586319, 0.0..."
1397,person_8,in/person_8/280.jpg,"[-0.08488425612449646, 0.10276811569929123, 0...."
1399,person_8,in/person_8/283.jpg,"[-0.10904248803853989, 0.11424244940280914, 0...."
1402,person_8,in/person_8/290.jpg,"[-0.10288207232952118, 0.07681678235530853, 0...."


In [218]:
magface["path"] = magface["path"].apply(lambda x: x[55:]).values
# magface["path"].apply(lambda x: x.split("09/")[1], axis = 0)
openvino["path"] = openvino["path"].apply(lambda x: x[55:]).values


In [223]:
database = openvino.merge(magface, how="inner",on="path")
database = database.merge(dlib_data, how="inner",on="path")

In [226]:
df = database
feature_column_mag = "feature_512"
feature_column_openvino = "features_openvino"
feature_column_dlib = "feature_dlib"


In [228]:
df = df[df["alignment_error"]<=16]

In [238]:
data_features_openvino = list(df[feature_column_openvino].values)
data_features_openvino = np.array([np.array(eval(feature)) for feature in data_features_openvino])

In [234]:
data_features_magface = list(df[feature_column_mag].values)
data_features_magface = np.array([np.array(eval(feature)) for feature in data_features_magface])

In [235]:
data_features_dlib = list(df[feature_column_dlib].values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])

In [230]:
data_features_dlib.shape

(397, 128)

In [240]:
data_features_openvino.shape

(397, 256)

In [241]:
from sklearn.manifold import TSNE
import numpy as np
from numpy import reshape
import seaborn as sns

In [244]:
import plotly.express as px

# scale and move the coordinates so they fit [0; 1] range
def scale_to_01_range(x):
    # compute the distribution range
    value_range = (np.max(x) - np.min(x))

    # move the distribution so that it starts from zero
    # by extracting the minimal value from all its values
    starts_from_zero = x - np.min(x)

    # make the distribution fit [0; 1] by dividing by its range
    return starts_from_zero / value_range

def get_tsn_results(data_features):
    
    tsne = TSNE(n_components=2, verbose=1, random_state=123)
    z = tsne.fit_transform(data_features) 
    # extract x and y coordinates representing the positions of the images on T-SNE plot
    tx = z[:, 0]
    ty = z[:, 1]
    tx = scale_to_01_range(tx)
    ty = scale_to_01_range(ty)
    
    return tx, ty


tx_openvino, ty_openvino = get_tsn_results(data_features_openvino)

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 397 samples in 0.000s...
[t-SNE] Computed neighbors for 397 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 397 / 397
[t-SNE] Mean sigma: 5.772091
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.360363
[t-SNE] KL divergence after 1000 iterations: 0.176958


In [243]:
tx_magface, ty_magface = get_tsn_results(data_features_magface)

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 397 samples in 0.000s...
[t-SNE] Computed neighbors for 397 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 397 / 397
[t-SNE] Mean sigma: 11.454814
[t-SNE] KL divergence after 250 iterations with early exaggeration: 57.606674
[t-SNE] KL divergence after 1000 iterations: 0.188760


In [242]:
tx_dlib, ty_dlib = get_tsn_results(data_features_dlib)

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/home/chaki/anaconda3/envs/gdsy/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 397 samples in 0.000s...
[t-SNE] Computed neighbors for 397 samples in 0.004s...
[t-SNE] Computed conditional probabilities for sample 397 / 397
[t-SNE] Mean sigma: 0.229672
[t-SNE] KL divergence after 250 iterations with early exaggeration: 58.372150
[t-SNE] KL divergence after 1000 iterations: 0.190844


In [248]:
tx_dlib.shape

(397,)

In [251]:
labels.shape

(397,)

In [249]:
labels = df["id_x"].values

In [252]:
data = {'tx_openvino':tx_openvino,
        'ty_openvino':ty_openvino,
        'tx_magface':tx_magface,
        'ty_magface':ty_magface,
        'tx_dlib':tx_dlib,
        'ty_dlib':ty_dlib,
        'labels': labels,
        'path' : df.path,
        'alignment_error': df.alignment_error
        }
 
# Create DataFrame
df = pd.DataFrame(data)
df.to_csv("images/in/database_withdlib.csv")

In [253]:
df

,tx_openvino,ty_openvino,tx_magface,ty_magface,tx_dlib,ty_dlib,labels,path,alignment_error
14,0.738425,0.421377,0.547369,0.425063,0.853096,0.212694,person_27,in/person_27/943.jpg,2.981036
35,0.738294,0.421478,0.547563,0.425731,0.853134,0.210870,person_27,in/person_27/951.jpg,7.899172
40,0.929831,0.949225,0.015253,0.956564,0.958723,0.693848,person_12,in/person_12/476.jpg,0.423276
41,0.878723,0.997069,0.075394,0.878168,0.979666,0.607342,person_12,in/person_12/494.jpg,7.251142
42,0.868548,0.955852,0.032346,0.913154,0.915972,0.620180,person_12,in/person_12/592.jpg,8.439944
...,...,...,...,...,...,...,...,...,...
887,0.487893,0.056109,0.248532,0.095126,0.236040,0.784722,person_19,in/person_19/711.jpg,9.788915
888,0.516289,0.066821,0.284068,0.097404,0.325242,0.734091,person_19,in/person_19/754.jpg,9.721998
889,0.510520,0.066601,0.293289,0.102158,0.325954,0.732747,person_19,in/person_19/761.jpg,4.983419
891,0.495220,0.008469,0.220532,0.061249,0.282643,0.740580,person_19,in/person_19/841.jpg,2.666573


In [63]:
np.arange(0,402).shape

(402,)

In [113]:
df[df['labels'] == 10]['labels'] = 3

In [254]:
df = df.reset_index()

In [255]:
from utils.utils import colors

In [256]:
df["labels"] = df["labels"].astype(str)

In [257]:
fig = px.scatter(df, x="tx_openvino", y="ty_openvino", color="labels", color_discrete_sequence=px.colors.qualitative.Dark24, hover_data=["index","labels", "path","alignment_error"])
fig.show()

In [258]:
fig = px.scatter(df, x="tx_dlib", y="ty_dlib", color="labels", color_discrete_sequence=px.colors.qualitative.Dark24, hover_data=["index","labels", "path","alignment_error"])
fig.show()

In [259]:
fig = px.scatter(df, x="tx_magface", y="ty_magface", color="labels", color_discrete_sequence=px.colors.qualitative.Dark24, hover_data=["index","labels", "path","alignment_error"])
fig.show()

In [260]:
imshow(df, 454)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [106]:
dlib_data = pd.read_csv("face_recognition_data.csv")
data_features_dlib = list(dlib_data.features_openvino.values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])
indexes = [len(value) != 0 for value in data_features_dlib]
dlib_data = dlib_data[indexes]
data_features_dlib = list(dlib_data.features_openvino.values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])


0                                                      []
1       [-0.07003957778215408, 0.07996471226215363, 0....
2       [-0.07282204926013947, 0.036366019397974014, 0...
3       [-0.06365539878606796, 0.054096899926662445, 0...
4       [-0.039636239409446716, 0.06792464107275009, 0...
                              ...                        
1406                                                   []
1407                                                   []
1408                                                   []
1409                                                   []
1410                                                   []
Name: features_openvino, Length: 1411, dtype: object

In [111]:
database = database.merge(dlib_data, how="inner",on="path")

/tmp/ipykernel_84718/213011151.py:1: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [119]:
indexes = [len(value) != 0 for value in data_features_dlib]

In [124]:
dlib_data = dlib_data[indexes]

In [125]:
data_features_dlib = list(dlib_data.features_openvino.values)
data_features_dlib = np.array([np.array(eval(feature)) for feature in data_features_dlib])


In [130]:
database = database.merge(dlib_data, how="inner",on="path")

In [131]:
database

,id_x,path,features_openvino_x,alignment_error,id_y,feature_512,Unnamed: 0,id,features_openvino_y
